In [23]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

In [24]:
# load the model
model = load_model("model.h5")

In [25]:
# load scaler and encoders

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

with open("one_hot_encoder_geo.pkl", "rb") as file:
    onehot_encoder_geo = pickle.load(file)


In [50]:
onehot_encoder_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [26]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [27]:
[input_data["Geography"]]

['France']

In [28]:
# trasnorming categorical data to numerical data
geo_encoded = onehot_encoder_geo.transform([[input_data["Geography"]]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out())

c:\Users\madhu\churn_modelling_ann\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [29]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [30]:
#turning key-value pair into dataframe

input_df = pd.DataFrame([input_data])

In [31]:
# Trasnforming values of gender column to 
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])

In [32]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [33]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [35]:
input_df = pd.concat([input_df.drop(columns=["Geography"]), geo_encoded_df], axis=1)

In [36]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [37]:
# transform all the columns
input_scaled = scaler.transform(input_df)

In [39]:
# predict churn

predict = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step


In [40]:
predict

array([[0.03150013]], dtype=float32)

In [43]:
pred_proba = predict[0][0]

In [44]:
pred_proba

np.float32(0.031500127)

In [45]:
if pred_proba > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
